In [1]:
from ndv import data, ArrayViewer
from ndv.models import DataWrapper, ChannelMode
import numpy as np
from cmap import Colormap
from typing import Any, TypeGuard, Hashable, Mapping, Sequence
from .. import li
.

In [2]:
data_arr = data.cells3d()

In [3]:
# zcyx
data_arr.shape, data_arr.dtype, data_arr.min(), data_arr.max()

((60, 2, 256, 256), dtype('uint16'), np.uint16(0), np.uint16(65532))

In [4]:
# label to idx
LI = {
 'z': 0,
 'c': 1,
 'y': 2,
 'x': 3,
}
# index to label
IL = dict((v,k) for k,v in LI.items())

Z = LI['z']
C = LI['c']
Y = LI['y']
X = LI['x']

In [5]:
class CustomWrapper(DataWrapper):
    PRIORITY = 10

    __li = {
     'z': 0,
     'c': 1,
     'y': 2,
     'x': 3,
    }

    @classmethod
    def supports(cls, obj: Any) -> TypeGuard[np.ndarray]:
        return isinstance(obj, np.ndarray)

    @property
    def dims(self) -> tuple[Hashable, ...]:
        return tuple(self.__li.keys())

    @property
    def coords(self) -> Mapping[Hashable, Sequence]:
        return {label: range(self._data.shape[idx]) for label, idx in self.__li.items()}

In [6]:
data_wrapper = CustomWrapper(data_arr)

In [7]:
viewer = ArrayViewer(data_wrapper, visible_axes=('y', 'x'), channel_axis='c', channel_mode=ChannelMode.COMPOSITE, default_lut={'cmap': Colormap('viridis')})

RFBOutputContext()

In [8]:
viewer.show()

In [12]:
# 3d
#viewer.display_model.visible_axes = (X,Y,X)
# 2d
#viewer.display_model.visible_axes = (Y,X)

In [ ]:
#viewer.display_model.channel_mode = "grayscale"

In [ ]:
#viewer.display_model.current_index.update({Y: 0})